## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,2021-10-19 15:56:41,-33.6500,115.3333,55.09,81,100,28.81,overcast clouds
1,1,Tuatapere,NZ,2021-10-19 15:57:51,-46.1333,167.6833,40.33,72,71,3.85,broken clouds
2,2,Puerto Ayora,EC,2021-10-19 15:57:49,-0.7393,-90.3518,75.16,79,72,9.48,broken clouds
3,3,Kaitangata,NZ,2021-10-19 15:54:34,-46.2817,169.8464,42.93,72,21,8.34,few clouds
4,4,Clyde River,CA,2021-10-19 15:57:23,70.4692,-68.5914,37.63,70,75,16.11,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                      (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Puerto Ayora,EC,2021-10-19 15:57:49,-0.7393,-90.3518,75.16,79,72,9.48,broken clouds
8,8,Saint-Philippe,RE,2021-10-19 15:56:59,-21.3585,55.7679,75.22,80,75,9.31,broken clouds
12,12,Atuona,PF,2021-10-19 15:56:47,-9.8000,-139.0333,77.59,77,37,21.63,light rain
19,19,Loandjili,CG,2021-10-19 16:02:46,-4.7561,11.8578,78.12,79,100,9.51,overcast clouds
41,41,Poum,NC,2021-10-19 15:53:45,-20.2333,164.0167,75.43,85,100,10.69,light rain
...,...,...,...,...,...,...,...,...,...,...,...
698,698,Almunecar,ES,2021-10-19 16:05:18,36.7339,-3.6907,88.72,76,77,8.19,broken clouds
701,701,Mogadishu,SO,2021-10-19 16:05:19,2.0371,45.3438,79.05,85,100,12.75,overcast clouds
702,702,Mumford,GH,2021-10-19 16:05:19,5.2618,-0.7590,80.96,77,89,14.41,overcast clouds
704,704,Bandarbeyla,SO,2021-10-19 15:59:06,9.4942,50.8122,79.18,85,6,12.68,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                223
City                   223
Country                222
Date                   223
Lat                    223
Lng                    223
Max Temp               223
Humidity               223
Cloudiness             223
Wind Speed             223
Current Description    223
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City","Country","Max Temp","Current Description","Lat","Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,
8,Saint-Philippe,RE,75.22,broken clouds,-21.3585,55.7679,
12,Atuona,PF,77.59,light rain,-9.8000,-139.0333,
19,Loandjili,CG,78.12,overcast clouds,-4.7561,11.8578,
41,Poum,NC,75.43,light rain,-20.2333,164.0167,
43,Namibe,AO,75.83,few clouds,-15.1961,12.1522,
46,Itarema,BR,89.28,broken clouds,-2.9248,-39.9167,
49,Victoria,HK,80.33,light rain,22.2855,114.1577,
51,Baguanos,CU,85.50,broken clouds,20.7517,-76.0269,
52,Hithadhoo,MV,82.81,broken clouds,-0.6000,73.0833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [9]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


### Find empty rows

In [10]:
hotel_df[hotel_df['Hotel Name'] == ""].index

Int64Index([57, 64, 92, 101, 186, 320, 337, 407, 472, 542, 629, 673], dtype='int64')

In [11]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

In [12]:
hotel_df[hotel_df['Hotel Name'] == ""].index

Int64Index([], dtype='int64')

In [13]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
              
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))